# Zomato Restaurant Data Analysis With Interactive Plots

![Zomato Logo](http://www.adageindia.in/photo/69351982/zomato-tells-inspirational-aspirational-stories-behind-red-t-shirts.jpg?112017)

**Zomato** is an Indian restaurant search and discovery service founded in 2008 by Deepinder Goyal and Pankaj Chaddah. It currently operates in 2 dozen countries.It provides information and reviews of restaurants, including images of menus where the restaurant does not have its own website and also facilitates online delivery.    

## About the dataset

1. **Restaurant Id**: Unique id of every restaurant across various cities of the world 
2. **Restaurant Name**: Name of the restaurant 
3. **Country Code**: Country in which restaurant is located 
4.   **City**: City in which restaurant is located 
5.  **Address**: Address of the restaurant 
6.   **Locality**: Location in the city 
7.   **Locality Verbose**: Detailed description of the locality 
8.   **Longitude**: Longitude coordinate of the restaurant's location 
9.  **Latitude**: Latitude coordinate of the restaurant's location 
10.   **Cuisines**: Cuisines offered by the restaurant 
11.   **Average Cost for two**: Cost for two people in different currencies 👫 
12.   **Currency**: Currency of the country • Has Table booking: yes/no 
13.   **Has Online delivery**: yes/ no 
14.   **Is delivering**: yes/ no 
15.   **Switch to order menu**: yes/no 
16.   **Price range**: range of price of food 
17.   **Aggregate Rating**: Average rating out of 5 
18.   **Rating color**: depending upon the average rating color 
19.   **Rating text**: text on the basis of rating of rating 
20.   **Votes**: Number of ratings casted by people

_________

## Importing Libraries and Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from wordcloud import WordCloud, STOPWORDS

import plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.offline as pyo
pyo.init_notebook_mode()
from plotly import tools

py.offline.init_notebook_mode(connected=True)

print('Done!')

In [ ]:
# Importing the dataset
zomato = pd.read_csv('../input/zomato.csv', encoding = "ISO-8859-1")

# Viewing 3 random sample from the dataframe
zomato.sample(3)

In [ ]:
# Size of the data
print('Shape of the df : ', zomato.shape)

#### Removing duplicates if any

In [ ]:
# dropping duplicates
zomato.drop_duplicates(subset = 'Restaurant ID', keep = 'first', inplace = True)

# Size of the data
zomato.shape

#### Replacing country codes to country names

In [ ]:
zomato.replace({
    'Country Code' : {
        1 : 'India',
        14 : 'Australia',
        30 : 'Brazil',
        37 : 'Canada',
        94 : 'Indonesia',
        148 : 'New Zealand',
        162 : 'Phillipines',
        166 : 'Qatar',
        184 : 'Singapore',
        189 : 'South Africa',
        191 : 'Sri Lanka',
        208 : 'Turkey',
        214 : 'UAE',
        215 : 'United Kingdom',
        216 : 'United States',
    }
}, inplace = True)

# renaming col country code to country 
zomato.rename(columns = {'Country Code' : 'Country'}, inplace = True)

Now, that we have converted `Country codes` to `Country names`, let's see the countries where zomato operates and also the number of restaurants in that country.

### Distribution of Restaurants of zomato (country wise)

In [ ]:
# creating a df 'za' to save country and no of restaurants in that country

# country name
x = zomato.Country.value_counts().index.values

# no. of restaurants in a particular country
y = zomato.Country.value_counts().values

# creating a df
za = pd.DataFrame({
    'Country': x,
    '# of Restaurants': y,
})

za.head(3)

In [ ]:
choropleth_map = dict (
    type = 'choropleth',
    locations = za['Country'],
    locationmode='country names',
    colorscale = 'Rainbow',
    z = za['# of Restaurants'],
    showscale = False,)

layout = go.Layout (
    title = go.layout.Title(
        text = "Zomato's presence on the planet"
    ),
)

fig = go.Figure(data = [choropleth_map], layout = layout)
py.offline.iplot(fig)

In [ ]:
# Of restaurants in India
nor_India = za[za['Country'] == 'India']['# of Restaurants'][0]

# of Indian restaurants
percent = nor_India / (za['# of Restaurants'].sum())

print('Percentage of indian restaurants is: ', percent * 100)

#### Inference :
**Zomato** has most number of restaurants in India(8.6K), followed by USA(434), UK(80) and UAE(60).    These numbers are not at all surprising as **Zomato** is an India based startup.

Since in our dataset about `90%` of data is of `zomato-India`, we will explore the data for Indian restaurants only.
____

### Preparing zomato_India dataset

In [ ]:
# Restaurants from India only
zomato_India = zomato[zomato.Country == 'India']

# Viewing 3 random samples
zomato_India.sample(3)

* Now that we have only `Indian` restaurants in our a data, we can drop the `Country`, `Currency` from the data set.
* Also we have see earlier that there were no duplicates, we can drop `Restaurant ID` as well.
* Also we can drop `Rating color` as the same info is given in the `Rating text` col.

In [ ]:
# Lets remove some noise from zomato_India dataset
zomato_India.drop(columns = ['Restaurant ID', 'Country', 'Currency', 'Rating color'], inplace = True)

# shape of dataset
print('Shape :', zomato_India.shape)

# 3 random samples
zomato_India.sample(3)

## EDA

Okay, now that we have our data for India ready and also we have `lat` and `lon` of all the restaurants, let's mark these restaurants on the `map` of India and see the spread of `zomato` across the nation.

In [ ]:
mapbox_access_token = 'pk.eyJ1IjoiYXZpa2FzbGl3YWwiLCJhIjoiY2p4MDhzYjI0MTg1bjQwcG05cjZqNjRtaiJ9.9MKul4M02Wp2TV3Fx-TwoQ'

In [ ]:
data = [
    go.Scattermapbox(
        lat = zomato_India['Latitude'],
        lon = zomato_India['Longitude'],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 5,
            color = '#cb202d'
        ),
        text = zomato_India['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize = True,
    hovermode = 'closest',
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = 26.52,
            lon = 78.37
        ),
        pitch = 1,
        zoom = 4,
        style = 'light'
    ),
    title = go.layout.Title (
        text = 'Zomato restaurants in India'
    ),
)

fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig, filename='Multiple Mapbox')

Panning around the map we can clearly see that, except for a few states `zomato` has its presence in all the major cities of the country.

Also, one can clearly see that the national capital region (`NCR`) has maximum number of restaurants tied up with `zomato`.

#### Top 10 Indian cities for zomato

In [ ]:
cities10_name = zomato_India.City.value_counts()[:9].index
cities10_value_log = np.log(zomato_India.City.value_counts()[:9])

data = [go.Bar(
            x = cities10_name,
            y = cities10_value_log,
            text = cities10_name,
            marker=dict(
                color=['red', 'orange',
               'green', 'blue','violet',
               'rgba(204,204,204,1)','rgba(204,204,204,1)',
               'rgba(204,204,204,1)','rgba(204,204,204,1)',
               'rgba(204,204,204,1)']),
        )]

layout = go.Layout(
    title = 'Top 10 citites for zomato',
    xaxis = dict(
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis = dict(
        title='Log10 of Number of Restaurant',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        ),
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
    )
)

fig = go.Figure(data = data, layout = layout)

pyo.iplot(fig, filename='color-bar')

#### Inference :
As seen in the map above, `zomato` has its presence in almost all the states of India, but `NCR` seems to be its fav and this fact is later proved from the barplot in which top 5 cities for `zomato` are from `NCR`.

Hence, so far we have established the fact that `India` is top market for `zomato` and in `India`, `NCR`'s market has a large presence of `zomato`

___

#### Segregating Zomato_India data as zomato_ncr and zomato_india (Not including NCR)

In [ ]:
# ncr
zomato_ncr = zomato_India.loc[(zomato_India.City).isin(['New Delhi','Gurgaon','Noida','Faridabad'])]

# rest of india
zomato_india = zomato_India.loc[~(zomato_India.City).isin(['New Delhi','Gurgaon','Noida','Faridabad'])]

### Does Price have an affect on rating?

#### Distribution of Avg price for 2 people  NCR & Non-NCR

In [ ]:
# Non NCR
zomato_india['Average Cost for two'].describe()

In [ ]:
zero = zomato_india[zomato_india['Average Cost for two'] == 0]
len(zero)

**Min** is `Rs.0` !!, I think, `0` here represents the absence of data. since only 9 values are missing, I am replacing 0 by the mean price

In [ ]:
# replacing 0 by mean price
zomato_india['Average Cost for two'].replace({
    0: zomato_India['Average Cost for two'].mean()
}, inplace = True)

zomato_india['Average Cost for two'].describe()

In [ ]:
# NCR
zomato_ncr['Average Cost for two'].describe()

In [ ]:
trace0 = go.Bar(
    x = zomato_india['Average Cost for two'].value_counts().index,
    y = np.log(zomato_india['Average Cost for two'].value_counts().values),
    text = (zomato_india['Average Cost for two'].value_counts().values),
)

layout = go.Layout(
    barmode ='group',
    shapes = [
        # Line reference to the axes
        {
            'type': 'line',
            'xref': 'x',
            'yref': 'y',
            'x0': zomato_india['Average Cost for two'].mean(),
            'y0': 0,
            'x1': zomato_india['Average Cost for two'].mean(),
            'y1': 8,
            'line': {
                'color': 'red',
                'width': 3,
                'dash': 'dashdot',
            },
        },
    ],
    annotations = [
        dict(
            x = zomato_india['Average Cost for two'].mean() + 5,
            y = 7,
            xref = 'x',
            yref = 'y',
            text = 'Average Price = 881.0 Rs',
            showarrow = True,
            arrowhead = 3,
            ax = 100,
            ay = 0,
        )
    ],
    title = go.layout.Title(
        text='India (Not including NCR)'  + '<br />' + 'Hover on bars for No. of Restaurants',
    ),
    xaxis = go.layout.XAxis(
        title = go.layout.xaxis.Title(
            text = 'Average Price in Rs. for Two People',
            font = dict(
                family = 'Courier New, monospace',
                size = 18,
                color = '#7f7f7f'
            )
        )
    ),
    yaxis = go.layout.YAxis(
        title = go.layout.yaxis.Title(
            text = 'Log10 of Number of restaurants',
            font = dict(
                family = 'Courier New, monospace',
                size = 18,
                color = '#7f7f7f'
            )
        )
    )
)

data = [trace0]

fig = go.Figure(data = data, layout = layout)
pyo.iplot(fig, filename='grouped-bar')


___

In [ ]:
trace0 = go.Bar(
    x = zomato_ncr['Average Cost for two'].value_counts().index,
    y = np.log(zomato_ncr['Average Cost for two'].value_counts().values),
    text = (zomato_ncr['Average Cost for two'].value_counts().values),
)

layout = go.Layout(
    barmode ='group',
    shapes = [
        # Line reference to the axes
        {
            'type': 'line',
            'xref': 'x',
            'yref': 'y',
            'x0': zomato_ncr['Average Cost for two'].mean(),
            'y0': 0,
            'x1': zomato_ncr['Average Cost for two'].mean(),
            'y1': 8,
            'line': {
                'color': 'red',
                'width': 3,
                'dash': 'dashdot',
            },
        },
    ],
    annotations = [
        dict(
            x = zomato_ncr['Average Cost for two'].mean() + 5,
            y = 7,
            xref = 'x',
            yref = 'y',
            text = 'Average Price = 600.0 Rs',
            showarrow = True,
            arrowhead = 3,
            ax = 100,
            ay = 0,
        )
    ],
    title = go.layout.Title(
        text='India (NCR Only)' + '<br />' + 'Hover on bars for No. of Restaurants',
    ),
    xaxis = go.layout.XAxis(
        title = go.layout.xaxis.Title(
            text = 'Average Price in Rs. for Two People',
            font = dict(
                family = 'Courier New, monospace',
                size = 18,
                color = '#7f7f7f'
            )
        )
    ),
    yaxis = go.layout.YAxis(
        title = go.layout.yaxis.Title(
            text = 'Log10 of Number of restaurants',
            font = dict(
                family = 'Courier New, monospace',
                size = 18,
                color = '#7f7f7f'
            )
        )
    )
)

data = [trace0]

fig = go.Figure(data = data, layout = layout)
pyo.iplot(fig, filename='grouped-bar')

#### Inference :
* 75% of restaurants in NCR cost about Rs.650 while for rest of India it is Rs.1200
* Average price for 2 in India is Rs.881 (Outside NCR), and Rs.600 (in NCR)
* Min is Rs. 100(Outside NCR) and Rs.5o (in NCR)
* Max is Rs. 8000(NCR) and Rs. 3600(Outside NCr)
* We can here conclude that there are more eco options @NCR on zomato than the rest of India.
___

### Price vs Rating

Here we will consider only the restaurants which have been rated and have atleast 50 votes.

In [ ]:
# Rated India (Not including NCR)
zomato_india_rated = zomato_india[zomato_india.Votes > 49]

# Rated India (NCR only)
zomato_ncr_rated = zomato_ncr[zomato_ncr.Votes > 49]

In [ ]:
pvr_india = go.Scatter(
    y = zomato_india_rated['Aggregate rating'],
    x = zomato_india_rated['Average Cost for two'],
    mode = 'markers',
    marker = dict(
        size = 6,
        color = zomato_india_rated['Aggregate rating'], #set color equal to a variable
        colorscale = 'Viridis',
        showscale = True
    ),
)

pvr_ncr = go.Scatter(
    y = zomato_ncr_rated['Aggregate rating'],
    x = zomato_ncr_rated['Average Cost for two'],
    mode = 'markers',
    marker = dict(
        size = 6,
        color = zomato_ncr_rated['Aggregate rating'], #set color equal to a variable
        colorscale = 'Viridis',
        showscale = False
    ),
)

fig = tools.make_subplots(rows = 1, cols = 2,subplot_titles=('Non-NCR', 'NCR'))

fig['layout']['yaxis1'].update(title = 'Rating (Outside NCR) on scale of 5')
fig['layout']['yaxis2'].update(title = 'Rating (NCR) on scale of 5')

fig['layout']['xaxis1'].update(title = 'Avg Price for 2')
fig['layout']['xaxis2'].update(title = 'Avg Price for 2')

fig.append_trace(pvr_india, 1, 1)
fig.append_trace(pvr_ncr, 1, 2)

fig['layout'].update(height = 600, width = 800, title = 'Price Vs Rating')
pyo.iplot(fig, filename = 'simple-subplot-with-annotations')

#### Inference :
* For restaurants outside NCR, we can say that restaurants have atleast a rating of 3, but in NCR we see all range of ratings.
* There is no linear relationship between price n ratings, as in for all price ranges the ratings are good n bad.

___

### What are the most value for money restaurants in NCR?

In [ ]:
zomato_ncr_votes = zomato_ncr_rated[zomato_ncr_rated.Votes >= 500]
zomato_ncr_rating = zomato_ncr_votes[zomato_ncr_votes['Aggregate rating'] > 3.9]
zomato_ncr_eco = zomato_ncr_rating[zomato_ncr_rating['Average Cost for two'] < 700]

In [ ]:
data = [
    go.Scattermapbox(
        lat = zomato_ncr_eco['Latitude'],
        lon = zomato_ncr_eco['Longitude'],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 5,
            color = '#cb202d'
        ),
        text = zomato_ncr_eco['Restaurant Name'] + '<br />' + zomato_ncr_eco['Locality Verbose'],
    )
]

layout = go.Layout(
    autosize = True,
    hovermode = 'closest',
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = 28.59,
            lon = 77.22,
        ),
        pitch = 1,
        zoom = 9,
        style = 'light'
    ),
    title = 'Best Value for money Restaurants in NCR'
)

fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig, filename='Multiple Mapbox')

#### Some of the best value for money bakeries in NCR

In [ ]:
zomato_ncr_bakery = zomato_ncr_eco.sort_values(by = 'Cuisines')[:5]

In [ ]:
data = [
    go.Scattermapbox(
        lat = zomato_ncr_bakery['Latitude'],
        lon = zomato_ncr_bakery['Longitude'],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 5,
            color = '#cb202d'
        ),
        text = zomato_ncr_bakery['Restaurant Name'] + '<br />' + zomato_ncr_bakery['Locality Verbose'],
    )
]

layout = go.Layout(
    autosize = True,
    hovermode = 'closest',
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = 28.59,
            lon = 77.22,
        ),
        pitch = 1,
        zoom = 9,
        style = 'light'
    ),
    title = 'Best Value for money bakeries in NCR'
)

fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig, filename='Multiple Mapbox')

#### Fast food chains' outlets in NCR

In [ ]:
fast_food_chains = ["Domino's Pizza", "Momo's King", 'Wow! India', "Dunkin' Donuts", 'Subway', "McDonald's",
                   "Pizza Hut", "Pizza Hut Delivery", "KFC", "Burger King", 'Chicago Pizza', 'Burger Point',
                   'Gopala', 'Yo! China', 'Ovenstory Pizza', 'RollsKing', 'Mad Over Donuts']

zomato_ncr_fast = zomato_ncr.loc[zomato_ncr['Restaurant Name'].isin(fast_food_chains)]

data = [
    go.Scattermapbox(
        lat = zomato_ncr_fast['Latitude'],
        lon = zomato_ncr_fast['Longitude'],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 5,
            color = '#cb202d'
        ),
        text = zomato_ncr_fast['Restaurant Name'] + '<br />' + zomato_ncr_fast['Locality Verbose'],
    )
]

layout = go.Layout(
    autosize = True,
    hovermode = 'closest',
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = 28.59,
            lon = 77.22,
        ),
        pitch = 1,
        zoom = 9,
    ),
    title = 'Fast Food Centers in NCR'
)

fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig, filename='Multiple Mapbox')

fast food centers are widely spread all over the NCR !
___

### What are the types of cuisines that zomato offers all over India ?

In [ ]:
# Download the image
# !wget --quiet https://www.redbytes.in/wp-content/uploads/2018/09/zomato-logo-AD6823E433-seeklogo.com_.png
    
# save mask to alice_mask
zomato_mask = np.array(Image.open('../input/zomato-logo-AD6823E433-seeklogo.com_.png'))

zomato_cuisines = []
for c in zomato_India['Cuisines']:
    listc = c.split(',')
    zomato_cuisines += listc
    
zomato_cuisines_text = ''
for text in zomato_cuisines:
    zomato_cuisines_text += (text + ' ')
    
stopwords = set(STOPWORDS)

# instantiate a word cloud object
zomato_wc = WordCloud(background_color='white', max_words=2000, mask=zomato_mask, stopwords=stopwords)

# generate the word cloud
zomato_wc.generate(zomato_cuisines_text)

# display the word cloud
fig = plt.figure()
fig.set_figwidth(8) # set width
fig.set_figheight(8) # set height

plt.imshow(zomato_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

**North Indian**, **Fast Food**, **Chinese**, **Mughlai**, **South Indian**, **Street Food** are the most famous cuisines that zomato offers to its coustmers in India.

____

### Table booking vs Rating

In [ ]:
zomato_india['Has Table booking'].describe()

In [ ]:
zomato_ncr['Has Table booking'].describe()

In [ ]:
x0 = zomato_ncr_rated['Has Table booking']
y0 = zomato_ncr_rated['Aggregate rating']

trace0 = go.Box(
    y = y0,
    x = x0,
    marker = dict(
        color = '#FF851B',
    ),
    boxmean=True,
    boxpoints = 'outliers',
)

layout = go.Layout(
    title = "Table Booking Vs Rating"
)

data = [trace0]
fig = go.Figure(data = data, layout = layout)
pyo.iplot(fig)

#### Inference :
* Restaurants with online table booking have slightly higher ratings in general.
* Most of the restaurants do not have online table booking facility.


### Where can I get awsm fine dine in NCR?
* Have online table booking facility.
* Rating of more than 4
* Rated by more than 200 people.

In [ ]:
top_fine_dine = zomato_ncr.loc[(zomato_ncr['Has Table booking'] == "Yes") & (zomato_ncr['Aggregate rating'] > 4) & (zomato_ncr['Votes'] > 200)].sort_values('Aggregate rating', ascending = False)

data = [
    go.Scattermapbox(
        lat = top_fine_dine['Latitude'],
        lon = top_fine_dine['Longitude'],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 5,
            color = '#cb202d'
        ),
        text = top_fine_dine['Restaurant Name'] + '<br />' + top_fine_dine['Locality Verbose'],
    )
]

layout = go.Layout(
    autosize = True,
    hovermode = 'closest',
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = 28.59,
            lon = 77.22,
        ),
        pitch = 1,
        zoom = 9,
    ),
    title = 'Top Fine Dines in NCR'
)

fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig, filename='Multiple Mapbox')

___

### Online delivery vs Rating

In [ ]:
zomato_india['Has Online delivery'].value_counts()

In [ ]:
zomato_ncr['Has Online delivery'].value_counts()

In [ ]:
x0 = zomato_india_rated['Has Online delivery']
y0 = zomato_india_rated['Aggregate rating']

trace0 = go.Box(
    y = y0,
    x = x0,
    marker = dict(
        color = '#3D9970',
    ),
    boxmean=True,
    boxpoints = 'outliers',
)

layout = go.Layout(
    title = "Online Delivery Vs Rating (India not including NCR)"
)

data = [trace0]
fig = go.Figure(data = data, layout = layout)
pyo.iplot(fig)

In [ ]:
x0 = zomato_ncr_rated['Has Online delivery']
y0 = zomato_ncr_rated['Aggregate rating']

trace0 = go.Box(
    y = y0,
    x = x0,
    marker = dict(
        color = '#FF4136',
    ),
    boxmean=True,
    boxpoints = 'outliers',
)
layout = go.Layout(
    title = "Online Delivery Vs Rating (India only NCR)"
)

data = [trace0]
fig = go.Figure(data = data, layout = layout)
pyo.iplot(fig)

#### Inference :
* Online booking **doesn't** impact the ratings of the restaurants for restaurants **in NCR**, but the it **does** for restaurants **outside NCR**.

___

### Do restaurants with more number of votes have lesser rating?

In [ ]:
pvr_india = go.Scatter(
    y = zomato_india_rated['Aggregate rating'],
    x = zomato_india_rated['Votes'],
    mode = 'markers',
    marker = dict(
        size = 4,
        color = zomato_india_rated['Aggregate rating'], #set color equal to a variable
        colorscale = 'Rainbow',
        showscale = True
    ),
)

pvr_ncr = go.Scatter(
    y = zomato_ncr_rated['Aggregate rating'],
    x = zomato_ncr_rated['Votes'],
    mode = 'markers',
    marker = dict(
        size = 4,
        color = zomato_ncr_rated['Aggregate rating'], #set color equal to a variable
        colorscale = 'Rainbow',
        showscale = False
    ),
)

fig = tools.make_subplots(rows = 1, cols = 2,subplot_titles=('Non-NCR', 'NCR'))

fig['layout']['yaxis1'].update(title = 'Rating (Outside NCR) on scale of 5')
fig['layout']['yaxis2'].update(title = 'Rating (NCR) on scale of 5')

fig['layout']['xaxis1'].update(title = 'Number of Votes')
fig['layout']['xaxis2'].update(title = 'Number of Votes')

fig.append_trace(pvr_india, 1, 1)
fig.append_trace(pvr_ncr, 1, 2)

fig['layout'].update(height = 600, width = 800, title = 'Votes Vs Rating')
pyo.iplot(fig, filename = 'simple-subplot-with-annotations')

#### Inference :
* Surprisingly it is the opposite of what I thought, for both outside and in NCR, restaurants with more than 4K votes have a rating of around 4 or more.

____

## Summary :

* Zomato has 90% of its eateries in India
* And in India most of them are from Delhi and nearby cities (NCR).
* Infact the top 5 citites for zomato in India are all from Delhi NCR.
* Average price for 2 is lower in NCR than other other parts of the nation.
    * One can even get a meal for Rs 50 in NCR
    * 75% of resturants of NCR serve for Rs. 650(for 2 or less) and outside NCR the number is Rs. 1200 for the same
* There was no linear relation found between price and rating, for all price range there were good and bad ratings.
* Among the many cuisines served by zomato, North Indian, Fast Food, Chinese, Mughlai, South Indian, Street Food are the most popular.
* There are lots of fast food centers in NCR.
* Not many restaurants have a facility of online table booking.
    * Restaurants that have online table booking have a slightly higher rating than the restaurants that do not offer online table booking.
* Ouside NCR, eateries with online delivery have higher rating than those who don't have online delivery.
    * But in NCR there is no affect on rating because of online delivery.
* Eateries with 4K+ votes have a rating greater than 4.